# Modeling Notebook

## Summary
In this Notebook:
- A first simple model is created
-
-

# First Simple Models

In [37]:
# Import statements
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier, Pool, metrics, cv

# Importing metrics function from functions.py
from functions import metrics as custom_score

In [7]:
# Load in cleaned data from last time.

# Training Data
X_train = pd.read_csv('../Data/train/X_train.csv', index_col=0)
y_train = pd.read_csv('../Data/train/y_train.csv', index_col=0)

# Testing Data
X_test = pd.read_csv('../Data/test/X_test.csv', index_col=0)
y_test = pd.read_csv('../Data/test/y_test.csv', index_col=0)


## Modeless Baseline
A modeless baseline is how accurate we would be if we guessed the majority class of our target variable. In this case, how accurate would we be if we guessed that the child in question did not have ADHD, for every child. This will be same for every split since we set the stratify parameter to true when we performed the TTS. It will be important to keep this metric in mind when modeling, to see when the model is just guessing the majority class.

In [14]:
# Getting % of each class and assigning to variables
no_adhd, adhd = y_train.value_counts(normalize=True)

# Printing modeless accuracy
print(f'If we said that each child in the set did not have ADHD, we would be {no_adhd*100:.0f}% accurate')


If we said that each child in the set did not have ADHD, we would be 90% accurate


## FSM - Sklearn Decision Tree

To start let's create the simplest model possible, to use as a baseline for future models. I'll use a Decision Tree model from Sklearn.

In [16]:
# Instantiating Tree
FSM_DT = DecisionTreeClassifier()

# Fitting Model
FSM_DT.fit(X_train, y_train)

# Score on the training data.
custom_score(y_train, FSM_DT.predict(X_train))

1.0

Wow! A perfect Score!
I guess we can go home, we did it, we solved ADHD

Just kidding, lets check the score on the testing data.

In [21]:
# Predictions
FSM_DT_preds = FSM_DT.predict(X_test)

# Print metrics
custom_score(y_test, FSM_DT_preds)

Accuracy: 0.90
Precision: 0.49
Recall: 0.54
F1: 0.51
ROC AUC: 0.74


In [25]:
# Checking the predictions to ensure that the model isn't guessing one class
pd.Series(FSM_DT_preds).value_counts()

2.0    7104
1.0     882
dtype: int64

## Analysis
The model is, obviously, overfit to the training data, and on the testing data it may as well be guessing. This is expected of an un-pruned DT model though.
I'd like to try a few different kinds of first models, so before iterating on this one let's create a few more FSMs.

## FSM - CatBoost
Catboost is not an Sklearn library, but is known for doing very well on categorical data like the one from this survey. Let's give it a shot and see how it does.

In [30]:
# Setting up the model
model = CatBoostClassifier(
    # Adding Accuracy as a metric
    custom_loss=[metrics.Accuracy()],
    random_seed=15,
    logging_level='Silent'
)

In [31]:
# Fitting the model to training data
model.fit(
    X_train, y_train,
    # Using X/y test as eval set
    eval_set=(X_test, y_test),
    # Plot the learning of the model
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [43]:
# Updating model params with Logloss function
cv_params = model.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
# Pooling data and cross validating
cv_data = cv(
    Pool(X_train, y_train),
    cv_params,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [44]:
# Printing training and testing scores.
print("Training Scores")
custom_score(y_train, model.predict(X_train))
print('\t')
print('Testing Scores')
custom_score(y_test, model.predict(X_test))

Training Scores
Accuracy: 0.97
Precision: 0.93
Recall: 0.73
F1: 0.82
ROC AUC: 0.86
	
Testing Scores
Accuracy: 0.94
Precision: 0.74
Recall: 0.56
F1: 0.64
ROC AUC: 0.77


### Analysis
Catboost has done quite well for a baseline model! There is, again, some bad overfitting occuring here. But the scores are still better then the base decision tree; precision, recall, and roc/auc have all vastly improved compared to the base decision tree. Lets move on to something totally different, a neural network